In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import warnings

# Ignore all warnings
warnings.filterwarnings("ignore")

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mallorn/sample_submission.csv
/kaggle/input/mallorn/test_log.csv
/kaggle/input/mallorn/train_log.csv
/kaggle/input/mallorn/split_17/train_full_lightcurves.csv
/kaggle/input/mallorn/split_17/test_full_lightcurves.csv
/kaggle/input/mallorn/split_01/train_full_lightcurves.csv
/kaggle/input/mallorn/split_01/test_full_lightcurves.csv
/kaggle/input/mallorn/split_02/train_full_lightcurves.csv
/kaggle/input/mallorn/split_02/test_full_lightcurves.csv
/kaggle/input/mallorn/split_08/train_full_lightcurves.csv
/kaggle/input/mallorn/split_08/test_full_lightcurves.csv
/kaggle/input/mallorn/split_04/train_full_lightcurves.csv
/kaggle/input/mallorn/split_04/test_full_lightcurves.csv
/kaggle/input/mallorn/split_07/train_full_lightcurves.csv
/kaggle/input/mallorn/split_07/test_full_lightcurves.csv
/kaggle/input/mallorn/split_15/train_full_lightcurves.csv
/kaggle/input/mallorn/split_15/test_full_lightcurves.csv
/kaggle/input/mallorn/split_20/train_full_lightcurves.csv
/kaggle/input/mallorn/

# Understanding Data

In [2]:
trainLog = pd.read_csv("/kaggle/input/mallorn/train_log.csv")
display(trainLog.head(10))
display(trainLog.info())


,object_id,Z,Z_err,EBV,SpecType,English Translation,split,target
0,Dornhoth_fervain_onodrim,3.0490,NaN,0.110,AGN,Trawn Folk (Dwarfs) + northern + Ents (people),split_01,0
1,Dornhoth_galadh_ylf,0.4324,NaN,0.058,SN II,Trawn Folk (Dwarfs) + tree + drinking vessel,split_01,0
2,Elrim_melethril_thul,0.4673,NaN,0.577,AGN,Elves + lover (fem.) + breath,split_01,0
3,Ithil_tobas_rodwen,0.6946,NaN,0.012,AGN,moon + roof + noble maiden,split_01,0
4,Mirion_adar_Druadan,0.4161,NaN,0.058,AGN,"jewel, Silmaril + father + Wild Man",split_01,0
5,Mirion_lalaith_neledh,1.1970,NaN,0.054,AGN,"jewel, Silmaril + laughter + enter, go in",split_01,0
6,Rochirrim_lebdas_pethron,0.1806,NaN,0.153,SN Ib,Horse.Lords (People) + index finger + narr...,split_01,0
7,acharn_lend_Mithrim,0.1887,NaN,0.051,AGN,"vengeance + journey, way + Sindar",split_01,0
8,achas_iar_gaur,1.2380,NaN,0.046,AGN,achas + blood + werewolf,split_01,0
9,aduial_nur_luith,2.2010,NaN,0.012,AGN,"evening, twilight + race + spell",split_01,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3043 entries, 0 to 3042
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   object_id            3043 non-null   object 
 1   Z                    3043 non-null   float64
 2   Z_err                0 non-null      float64
 3   EBV                  3043 non-null   float64
 4   SpecType             3043 non-null   object 
 5   English Translation  3043 non-null   object 
 6   split                3043 non-null   object 
 7   target               3043 non-null   int64  
dtypes: float64(3), int64(1), object(4)
memory usage: 190.3+ KB


None

In [3]:
testLog = pd.read_csv("/kaggle/input/mallorn/test_log.csv")
display(testLog.head(10))
display(testLog.info())


,object_id,Z,Z_err,EBV,SpecType,English Translation,split
0,Eluwaith_Mithrim_nothrim,0.5393,0.03013,0.610,NaN,Sindar + Sindar + family,split_01
1,Eru_heledir_archam,0.7282,0.03508,0.058,NaN,God + kingfisher (bird) + throne,split_01
2,Gonhir_anann_fuin,0.6026,0.03185,0.070,NaN,"Master of Stone, Dwarf + long, for a long tim...",split_01
3,Gwathuirim_haradrim_tegilbor,0.5298,0.03013,0.014,NaN,Shadowy People (of Dunland) + People of the S...,split_01
4,achas_minai_maen,0.1654,0.02304,0.027,NaN,"achas + distinct, unique, single + clever, sk...",split_01
5,adab_fae_gath,0.7192,0.03455,0.011,NaN,"building, house + soul, spirit + cavern",split_01
6,adel_draug_gaur,0.3024,0.02681,0.050,NaN,behind + wolf + werewolf,split_01
7,aderthad_cuil_galadhrim,0.1603,0.02390,0.036,NaN,"reunion + life + people of the Trees, Elves of...",split_01
8,aegas_laug_ithildin,0.7760,0.03543,0.081,NaN,mountain peak + warm + magic metal that only ...,split_01
9,aegas_mereth_law,0.3107,0.02616,0.026,NaN,"mountain peak + feast, festival + no! don't!",split_01


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7135 entries, 0 to 7134
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   object_id            7135 non-null   object 
 1   Z                    7135 non-null   float64
 2   Z_err                7135 non-null   float64
 3   EBV                  7135 non-null   float64
 4   SpecType             0 non-null      float64
 5   English Translation  7135 non-null   object 
 6   split                7135 non-null   object 
dtypes: float64(4), object(3)
memory usage: 390.3+ KB


None

In [4]:
# importing one train sample split
sampSplitTrain = pd.read_csv("/kaggle/input/mallorn/split_01/train_full_lightcurves.csv")
display(sampSplitTrain.head(10))
display(sampSplitTrain.info())

,object_id,Time (MJD),Flux,Flux_err,Filter
0,Dornhoth_fervain_onodrim,63314.4662,-1.630159,0.365777,z
1,Dornhoth_fervain_onodrim,63780.9674,10.499389,0.253867,r
2,Dornhoth_fervain_onodrim,63789.7693,5.866250,1.559241,y
3,Dornhoth_fervain_onodrim,63794.1702,3.903623,0.376854,r
4,Dornhoth_fervain_onodrim,63794.1702,5.226644,0.516864,i
5,Dornhoth_fervain_onodrim,63824.9769,0.984804,0.109919,g
6,Dornhoth_fervain_onodrim,63824.9769,1.485012,0.181529,r
7,Dornhoth_fervain_onodrim,63829.3779,0.170766,0.551879,u
8,Dornhoth_fervain_onodrim,63829.3779,1.183222,0.208419,r
9,Dornhoth_fervain_onodrim,63842.5807,0.725684,0.533212,i


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26324 entries, 0 to 26323
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   object_id   26324 non-null  object 
 1   Time (MJD)  26324 non-null  float64
 2   Flux        26313 non-null  float64
 3   Flux_err    26324 non-null  float64
 4   Filter      26324 non-null  object 
dtypes: float64(3), object(2)
memory usage: 1.0+ MB


None

In [5]:
# importing one test sample split
sampSplitTest = pd.read_csv("/kaggle/input/mallorn/split_01/test_full_lightcurves.csv")
display(sampSplitTest.head(10))
display(sampSplitTest.info())

,object_id,Time (MJD),Flux,Flux_err,Filter
0,Elrim_sador_hun,61120.3730,-0.867298,1.436325,z
1,Elrim_sador_hun,62428.9395,-0.412872,0.972737,y
2,Elrim_sador_hun,62431.2074,0.807585,0.421561,i
3,Elrim_sador_hun,62431.2074,0.564190,0.702448,z
4,Elrim_sador_hun,62444.8147,0.495302,0.135575,r
5,Elrim_sador_hun,62449.3504,0.661649,0.231340,i
6,Elrim_sador_hun,62449.3504,-0.165556,0.411221,z
7,Elrim_sador_hun,62428.9395,0.182961,0.550523,z
8,Elrim_sador_hun,62608.1020,-0.005976,1.268166,y
9,Elrim_sador_hun,62648.9238,-0.296071,0.202261,r


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59235 entries, 0 to 59234
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   object_id   59235 non-null  object 
 1   Time (MJD)  59235 non-null  float64
 2   Flux        59212 non-null  float64
 3   Flux_err    59235 non-null  float64
 4   Filter      59235 non-null  object 
dtypes: float64(3), object(2)
memory usage: 2.3+ MB


None

#  EDA

In [6]:
import matplotlib.pyplot as plt
import plotly.express as px

In [7]:
trainLog.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3043 entries, 0 to 3042
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   object_id            3043 non-null   object 
 1   Z                    3043 non-null   float64
 2   Z_err                0 non-null      float64
 3   EBV                  3043 non-null   float64
 4   SpecType             3043 non-null   object 
 5   English Translation  3043 non-null   object 
 6   split                3043 non-null   object 
 7   target               3043 non-null   int64  
dtypes: float64(3), int64(1), object(4)
memory usage: 190.3+ KB


In [8]:
trainLog.describe()

,Z,Z_err,EBV,target
count,3043.000000,0.0,3043.000000,3043.000000
mean,0.670674,NaN,0.055547,0.048636
std,0.539269,NaN,0.061349,0.215142
min,0.008771,NaN,0.002000,0.000000
25%,0.317750,NaN,0.020000,0.000000
50%,0.481800,NaN,0.037000,0.000000
75%,0.883550,NaN,0.069000,0.000000
max,4.924000,NaN,0.758000,1.000000


In [9]:
# Examine Z value w.r.t the target.
# Question : Do TDEs tend to occur at different distances compared to other transients?

fig = px.histogram(trainLog, x="Z")
fig.show()
